The client ABC is a US based large retailer of apparel and accessories. ABC management is planning to expand their business by opening more shops in different locations across the US.

In [78]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage
import os
from io import StringIO
sns.set()


In [79]:
df=pd.read_excel("Data2.xlsx",sheet_name="Data")
df.head()

,Store ID,Pop class,Sales,Centre Type,Climate,Total Store Sq Ft,Sales Floor Sq Ft,Girls Sq Ft,Guys Sq Ft,Accessories Sq Ft,...,Poverty: In poverty,Male 14 to 35 CrYr,Female 14 to 35 CrYr,Total Pop 14 to 35 CrYr,Under 25 Low Income,Under 25 Med Income,Under 25 High Income,25 to 44 Low Income,25 to 44 Med Income,25 to 44 High Income
0,1,Suburban,1345251.88,Outlet,Frigid,4735.0,4015.6,2032.3,947.7,747.5,...,1680,4444,4095,8539,74,285,7,683,1692,1291
1,2,Urban,1669906.40,Mall,Moderate,8068.0,7128.6,2735.2,1788.1,1251.3,...,21003,58486,52872,111358,1317,1396,358,11104,21417,20357
2,3,Urban,2528042.74,Outlet,Hot,4583.2,3961.7,2082.9,1030.2,551.7,...,107701,286481,275998,562479,14977,11109,2870,63228,120359,99407
3,4,Suburban,1281613.45,Outlet,Moderate,5772.3,4713.0,2140.1,1149.0,1162.2,...,23477,45764,43429,89193,5135,2426,512,10011,15485,9238
4,5,Urban,1278092.87,Outlet,Cold,3503.0,3239.0,1870.6,769.5,483.5,...,65968,157166,152880,310046,11249,6220,2311,32448,57978,74382


In [80]:
df.drop(columns=["Store ID"], inplace=True)
df.dropna(inplace=True)
df = pd.get_dummies(df, drop_first=True)


In [81]:
X=df.drop("Sales",axis=1)
y=df["Sales"]
X=X.apply(pd.to_numeric).astype(float)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

selector = VarianceThreshold(threshold=0.01)
X_train = pd.DataFrame(selector.fit_transform(X_train), columns=X_train.columns[selector.get_support()])
X_test = X_test[X_train.columns]

In [83]:
#correlation matrix
plt.figure(figsize=(18, 14))
corr_matrix = X_train.corr()

sns.heatmap(
    corr_matrix,
    cmap="coolwarm",
    annot=False,
    square=True,
    cbar_kws={'shrink': 0.75},
    linewidths=0.5,
    xticklabels=True,
    yticklabels=True
)

plt.xticks(rotation=45, ha='right', fontsize=8)
plt.yticks(fontsize=8)
plt.title("Feature Correlation Matrix Before Removal", fontsize=14, fontweight='bold')
plt.tight_layout()

correlation = "correlation_matrix.png"
plt.savefig(correlation, dpi=300)
plt.close()


In [84]:
#Addressing Multicollinearity
def multicollinearity(X,threshold=0.9):
    correlation=X.corr().abs()
    upper = correlation.where(np.triu(np.ones(correlation.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(f"Dropping due to high correlation: {to_drop}")
    return X.drop(columns=to_drop), to_drop
X_train_reduced, dropped_corr_features = multicollinearity(X_train)
X_test = X_test[X_train_reduced.columns]

Dropping due to high correlation: ['Sales Floor Sq Ft', 'Sex Males CrYr', 'Sex Females CrYr', 'Householder under 25 years:', 'Householder 25 to 44 years:', 'Family Inc: Aggregate Average', 'Wealth: Average Household', 'Poverty: In poverty', 'Male 14 to 35 CrYr', 'Female 14 to 35 CrYr', 'Total Pop 14 to 35 CrYr', 'Under 25 Low Income', 'Under 25 Med Income', 'Under 25 High Income', '25 to 44 Low Income', '25 to 44 Med Income', '25 to 44 High Income', 'Girls Segmentation_On Trend']


In [85]:
X_train_sm=sm.add_constant(X_train_reduced.reset_index(drop=True))
y_train_reset=y_train.reset_index(drop=True)
model=sm.OLS(y_train_reset,X_train_sm).fit()

In [86]:
X_test_sm=sm.add_constant(X_test.reset_index(drop=True))
y_test_reset=y_test.reset_index(drop=True)
pred_model=model.predict(X_test_sm)



In [87]:
mape = np.mean(np.abs((y_test_reset - pred_model) / y_test_reset)) * 100
mae = mean_absolute_error(y_test_reset, pred_model)
rmse = np.sqrt(mean_squared_error(y_test_reset, pred_model))
r2 = r2_score(y_test_reset,pred_model)
performance_summary = pd.DataFrame({
    'Metric': ['MAPE (%)', 'MAE', 'RMSE', 'R-squared'],
    'Value': [round(mape, 2), round(mae, 2), round(rmse, 2), round(r2, 4)]
})

In [88]:
html_table = model.summary().tables[1].as_html()
coef_summary = pd.read_html(StringIO(html_table), header=0, index_col=0)[0]


In [89]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test_reset, y=pred_model, color='blue', s=60)
plt.plot([y_test_reset.min(), y_test_reset.max()], [y_test_reset.min(), y_test_reset.max()],'r--')
plt.xlabel("Actual Sales")
plt.ylabel("Predicted Sales")
plt.title("Actual vs Predicted Sales")
plt.tight_layout()
plot1 = "actual_vs_predicted.png"
plt.savefig(plot1)
plt.close()


In [90]:
residuals = y_test_reset - pred_model
plt.figure(figsize=(8, 6))
sns.residplot(x=pred_model, y=residuals, lowess=True, color="purple", line_kws={'color': 'red'})
plt.xlabel("Predicted Sales")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.tight_layout()
plot2 = "residual_plot.png"
plt.savefig(plot2)
plt.close()


In [91]:
model_summary= model.summary().as_text()
model_summary_df = pd.DataFrame(model_summary.splitlines(), columns=["Model Summary"])


In [92]:
excel= "Model Report.xlsx"
with pd.ExcelWriter(excel, engine="xlsxwriter") as writer:
    performance_summary.to_excel(writer, sheet_name="Model Performance", index=False)
    coef_summary.to_excel(writer, sheet_name="Model Coefficients")
    pd.DataFrame({'Dropped_Features': dropped_corr_features}).to_excel(writer, sheet_name="Dropped Correlated")
    model_summary_df.to_excel(writer, sheet_name="OLS Summary", index=False)

In [93]:
wb = load_workbook(excel)
ws = wb["Model Performance"]

img1 = XLImage(plot1)
img1.width, img1.height = 480, 360
ws.add_image(img1, "E5")

img2 = XLImage(plot2)
img2.width, img2.height = 480, 360
ws.add_image(img2, "E25")

img3 = XLImage(correlation)
img3.width, img3.height = 600, 480
ws.add_image(img3, "E45")

wb.save(excel)

os.remove(plot1)
os.remove(plot2)
os.remove(correlation)

print(f"Report saved as: {excel}")


Report saved as: Model Report.xlsx
